<a href="https://colab.research.google.com/github/rshaikh95/Diabetes_Dataset/blob/main/Diabetic_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Commands to install some of the libraries in-case if they are not installed
# Any other library that needs to be installed just use: !pip install <library name>
# !pip install seaborn
# !pip install missingno
# !pip install xgboost
# !pip install catboost
# !pip install regex
# !pip install sklearn
# !pip install pandas
# !pip install numpy
# !pip install imblearn
# !pip install lightgbm

### Import Libraries

In [145]:
import pandas as pd   # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np   # linear algebra
import matplotlib.pyplot as plt  #graphs and plots
import seaborn as sns   #data visualizations
import csv # Some extra functionalities for csv  files - reading it as a dictionary
from lightgbm import LGBMClassifier #sklearn is for machine learning and statistical modeling including classification, regression, clustering and dimensionality reduction

from sklearn.model_selection import train_test_split, cross_validate   #break up dataset into train and test sets

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler


# importing python library for working with missing data
import missingno as msno
# To install missingno use: !pip install missingno
import re    # This library is used to perform regex pattern matching

# import various functions from sklearn
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
import xgboost as xgb
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, classification_report, make_scorer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split

### Read CSV File

In [146]:
diabetes = pd.read_csv('diabetic_data.csv')

In [ ]:
diabetes.head(n=50)

### Y Values: Readmitted

In [ ]:
y = diabetes['readmitted']
print(f'Percentage of patient readmitted for diabetes (>30 days): % {round(y.value_counts(normalize=True)[1]*100,2)} --> ({y.value_counts()[1]} patient)\nPercentage of patient not readmitted for diabetes: % {round(y.value_counts(normalize=True)[0]*100,2)} --> ({y.value_counts()[0]} patient)\nPercentage of patient readmitted for diabetes (<30 days): % {round(y.value_counts(normalize=True)[2]*100,2)} --> ({y.value_counts()[2]} patient)')

### Figure of Diabetes Readmitted ( NO , <30 , >30)

In [ ]:
fig = px.histogram(diabetes, x="readmitted", title='Diabetes', width=400, height=400)
fig.show()

In [ ]:
 diabetes.info()

## Defined missing data


In [ ]:
diabetes.duplicated().sum()

Found "?" to be missing value: replaced with NA

In [ ]:
diabetes.replace("?", np.nan, inplace=True)
diabetes

In [ ]:
def missing (diabetes):
    missing_number = diabetes.isnull().sum().sort_values(ascending=False)
    missing_percent = (diabetes.isnull().sum()/diabetes.isnull().count()).sort_values(ascending=False)
    missing_values = pd.concat([missing_number, missing_percent], axis=1, keys=['Missing_Number', 'Missing_Percent'])
    return missing_values


missing(diabetes)

In [154]:

diabetes.drop(['weight'], axis = 1, inplace = True)
# dropping weight due to 96% missing

In [ ]:
def missing (diabetes):
    missing_number = diabetes.isnull().sum().sort_values(ascending=False)
    missing_percent = (diabetes.isnull().sum()/diabetes.isnull().count()).sort_values(ascending=False)
    missing_values = pd.concat([missing_number, missing_percent], axis=1, keys=['Missing_Number', 'Missing_Percent'])
    return missing_values


missing(diabetes)

In [ ]:
diabetes.dropna(inplace=True)
diabetes
# dropped all NA values

### Categorical and Numerical variables

In [157]:
categorical = ['race', 'gender', 'age', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
               'payer_code', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3',
                'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
               'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone',
              'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide',
              'examide', 'citoglipton', 'insulin', 'glyburide-metformin', 'glipizide-metformin',
               'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone',
               'change', 'diabetesMed', 'readmitted']

numerical = ['time_in_hospital','num_lab_procedures', 'num_procedures','num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses']

In [ ]:
diabetes[numerical].describe()

## Skewness

In [ ]:
diabetes[numerical].skew()

Based on skewness, num of lab procedures and number of diagnoses has a slight left skew (negative) while the rest have right skew (positive)

## Univariate Analysis

In [ ]:
diabetes[numerical].hist(figsize=(20,10));

Categorical Variables:

               'payer_code', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3',
                'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
               'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone',
              'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide',
              'examide', 'citoglipton', 'insulin', 'glyburide-metformin', 'glipizide-metformin',
               'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone',
               'change', 'diabetesMed', 'readmitted

In [ ]:
categorical = ['race', 'gender', 'age', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
               'payer_code', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3',
                'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
               'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone',
              'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide',
              'examide', 'citoglipton', 'insulin', 'glyburide-metformin', 'glipizide-metformin',
               'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone',
               'change', 'diabetesMed', 'readmitted']
# loop for each column instead of repeating code
for column in categorical:
 value_counts = diabetes[column].value_counts(normalize=True)
 # Printing the value counts
 print(f'{column}\n{value_counts}\n')
 # Creating a histogram using Plotly Express
 fig = px.histogram(diabetes, x=column, title=column, width=500, height=500)
 fig.show()

In [ ]:
print (f'{round(diabetes["gender"].value_counts(normalize=True)*100,2)}')
fig = px.histogram(diabetes, x="gender", title='Gender', width=500, height=500)
fig.show()

In [ ]:
print (f'{round(diabetes["race"].value_counts(normalize=True)*100,2)}')
fig = px.histogram(diabetes, x="race", title='Race', width=500, height=500)
fig.show()

In [170]:
print (f'{round(diabetes["age"].value_counts(normalize=True)*100,2)}')
fig = px.histogram(diabetes, x="age", title='Age', width=500, height=500)
fig.show()

[70-80)     25.27
[60-70)     22.28
[80-90)     17.80
[50-60)     16.68
[40-50)      9.29
[30-40)      3.46
[90-100)     3.14
[20-30)      1.82
[10-20)      0.25
[0-10)       0.01
Name: age, dtype: float64


In [ ]:
print (f'{round(diabetes["max_glu_serum"].value_counts(normalize=True)*100,2)}')
fig = px.histogram(diabetes, x="max_glu_serum", title='Max Glucose Serum', width=500, height=500)
fig.show()

In [179]:
print (f'{round(diabetes["A1Cresult"].value_counts(normalize=True)*100,2)}')
fig = px.histogram(diabetes, x="A1Cresult", title='A1C Results', width=500, height=500)
fig.show()

None    84.43
>8       7.23
Norm     4.24
>7       4.10
Name: A1Cresult, dtype: float64


In [ ]:
print (f'{round(diabetes["metformin"].value_counts(normalize=True)*100,2)}')
fig = px.histogram(diabetes, x="metformin", title='Metformin', width=500, height=500)
fig.show()

In [ ]:
print (f'{round(diabetes["readmitted"].value_counts(normalize=True)*100,2)}')
fig = px.histogram(diabetes, x="readmitted", title='Readmitted', width=500, height=500)
fig.show()

## Bivariate Analysis

In [163]:
diabetes["readmitted"].unique()

array(['NO', '>30', '<30'], dtype=object)

In [164]:
# Replacing values
diabetes["readmitted"] = diabetes["readmitted"].replace({"NO": 0, "<30": 1, ">30": 2})

In [165]:
diabetes["readmitted"].value_counts()

0    15154
2     8744
1     2857
Name: readmitted, dtype: int64

## Race and Readmission

In [176]:
print (f'A person with Caucasian race has a probability of {round(diabetes[diabetes["race"]=="Caucasian"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person with African American race has a probability of {round(diabetes[diabetes["race"]=="AfricanAmerican"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person with Hispanic race has a probability of {round(diabetes[diabetes["race"]=="Hispanic"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person with Asian race has a probability of {round(diabetes[diabetes["race"]=="Asian"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person with Other race has a probability of {round(diabetes[diabetes["race"]=="Other"]["readmitted"].mean()*100,2)} % to be readmitted')

A person with Caucasian race has a probability of 78.49 % to be readmitted

A person with African American race has a probability of 67.97 % to be readmitted

A person with Hispanic race has a probability of 68.39 % to be readmitted

A person with Asian race has a probability of 70.8 % to be readmitted

A person with Other race has a probability of 73.57 % to be readmitted


In [ ]:
fig = px.histogram(diabetes, x="race", color="readmitted",width=600, height=600)
fig.show()

## Gender and Readmission

In [168]:
print (f'A person with Male gender has a probability of {round(diabetes[diabetes["gender"]=="Male"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person with Female gender has a probability of {round(diabetes[diabetes["gender"]=="Female"]["readmitted"].mean()*100,2)} % to be readmitted')

A person with Male gender has a probability of 74.78 % to be readmitted

A person with Female gender has a probability of 77.1 % to be readmitted


In [ ]:
fig = px.histogram(diabetes, x="gender", color="readmitted",width=600, height=600)
fig.show()


## Age and Readmission

In [ ]:
print (f'A person [70-80) has a probability of {round(diabetes[diabetes["age"]=="[70-80)"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person [60-70) has a probability of {round(diabetes[diabetes["age"]=="[60-70)"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person [80-90) has a probability of {round(diabetes[diabetes["age"]=="[80-90)"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person [50-60) has a probability of {round(diabetes[diabetes["age"]=="[50-60)"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person [40-50) has a probability of {round(diabetes[diabetes["age"]=="[40-50)"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person [30-40) has a probability of {round(diabetes[diabetes["age"]=="[30-40)"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person [90-100) has a probability of {round(diabetes[diabetes["age"]=="[90-100)"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person [20-30) has a probability of {round(diabetes[diabetes["age"]=="[20-30)"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person [10-20) has a probability of {round(diabetes[diabetes["age"]=="[10-20)"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person [0-10) has a probability of {round(diabetes[diabetes["age"]=="[0-10)"]["readmitted"].mean()*100,2)} % to be readmitted')

In [ ]:
fig = px.histogram(diabetes, x="age", color="readmitted",width=600, height=600)
fig.show()


## Metformin and Readmission

In [177]:
print (f'A person with No metformin has a probability of {round(diabetes[diabetes["metformin"]=="No"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person with Steady metformin has a probability of {round(diabetes[diabetes["metformin"]=="Steady"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person with Up metformin has a probability of {round(diabetes[diabetes["metformin"]=="Up"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person with Down metformin has a probability of {round(diabetes[diabetes["metformin"]=="Down"]["readmitted"].mean()*100,2)} % to be readmitted')


A person with No metformin has a probability of 76.52 % to be readmitted

A person with Steady metformin has a probability of 74.36 % to be readmitted

A person with Up metformin has a probability of 74.07 % to be readmitted

A person with Down metformin has a probability of 69.66 % to be readmitted


In [ ]:
fig = px.histogram(diabetes, x="metformin", color="readmitted",width=600, height=600)
fig.show()

## A1C Results and Readmission

In [180]:
print (f'A person with None A1C result has a probability of {round(diabetes[diabetes["A1Cresult"]=="None"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person with >8 A1C result has a probability of {round(diabetes[diabetes["A1Cresult"]==">8"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person with Norm A1C result has a probability of {round(diabetes[diabetes["A1Cresult"]=="Norm"]["readmitted"].mean()*100,2)} % to be readmitted')

print()

print (f'A person with >7 A1C result has a probability of {round(diabetes[diabetes["A1Cresult"]==">7"]["readmitted"].mean()*100,2)} % to be readmitted')


A person with None A1C result has a probability of 76.79 % to be readmitted

A person with >8 A1C result has a probability of 73.47 % to be readmitted

A person with Norm A1C result has a probability of 65.7 % to be readmitted

A person with >7 A1C result has a probability of 75.77 % to be readmitted


In [ ]:
fig = px.histogram(diabetes, x="A1Cresult", color="readmitted",width=600, height=600)
fig.show()

## Metrics of Importance

In [ ]:
from sklearn.metrics import mutual_info_score
def cat_mut_inf(series):
    return mutual_info_score(series, diabetes['readmitted'])

diabetes_cat = diabetes[categorical].apply(cat_mut_inf)
diabetes_cat = diabetes_cat.sort_values(ascending=False).to_frame(name='mutual_info_score')
diabetes_cat

## Correlation Matrix & Scatter Plots


In [197]:
diabetes[numerical].corr()

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
time_in_hospital,1.000000,0.295395,0.131720,0.456300,-0.012761,-0.006117,0.058222,0.228072
num_lab_procedures,0.295395,1.000000,0.009714,0.265501,-0.014817,0.004575,0.023095,0.139283
num_procedures,0.131720,0.009714,1.000000,0.332057,0.017860,-0.038181,-0.052389,0.056218
num_medications,0.456300,0.265501,0.332057,1.000000,0.051744,0.011401,0.064613,0.249727
number_outpatient,-0.012761,-0.014817,0.017860,0.051744,1.000000,0.068830,0.109317,0.088650
number_emergency,-0.006117,0.004575,-0.038181,0.011401,0.068830,1.000000,0.327175,0.063548
number_inpatient,0.058222,0.023095,-0.052389,0.064613,0.109317,0.327175,1.000000,0.125628
number_diagnoses,0.228072,0.139283,0.056218,0.249727,0.088650,0.063548,0.125628,1.000000


In [ ]:
diabetes.groupby('readmitted')[numerical].mean()

In [ ]:
numerical = ['time_in_hospital','num_lab_procedures', 'num_procedures','num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses']
diabetes[numerical].corr()

In [ ]:
 fig = px.scatter(diabetes, x='time_in_hospital', y='num_lab_procedures', title='Time in Hospital vs Num Lab Procedures')
fig.show()

In [ ]:
fig = px.scatter(diabetes, x='time_in_hospital', y='num_medications', title='Time in Hospitals vs Num of Medication')
fig.show()

In [ ]:

fig = px.scatter(diabetes, x='time_in_hospital', y='number_outpatient', title='Time in Hospital vs Num Outpatient')
fig.show()

In [ ]:
fig = px.scatter(diabetes, x='time_in_hospital', y='number_emergency', title='Time in Hospital vs Num Emergency')
fig.show()

In [ ]:
fig = px.scatter(diabetes, x='time_in_hospital', y='number_inpatient', title='Time in Hospital vs Num Inpatient')
fig.show()

In [ ]:
fig = px.scatter(diabetes, x='time_in_hospital', y='number_diagnoses', title='Time in Hospital vs Num Diagnoses')
fig.show()